In [8]:
import pandas as pd
import numpy as np
import datetime
import random
import os
import sys
import holidays


import sklearn
from sklearn.ensemble import VotingRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error


import xgboost
from xgboost import XGBRegressor
import catboost
from catboost import CatBoostRegressor

In [3]:
train = pd.read_csv("./train.csv")
test = pd.read_csv("./test.csv")
train

,ID,timestamp,item,corporation,location,supply(kg),price(원/kg)
0,TG_A_J_20190101,2019-01-01,TG,A,J,0.0,0.0
1,TG_A_J_20190102,2019-01-02,TG,A,J,0.0,0.0
2,TG_A_J_20190103,2019-01-03,TG,A,J,60601.0,1728.0
3,TG_A_J_20190104,2019-01-04,TG,A,J,25000.0,1408.0
4,TG_A_J_20190105,2019-01-05,TG,A,J,32352.0,1250.0
...,...,...,...,...,...,...,...
59392,RD_F_J_20230227,2023-02-27,RD,F,J,452440.0,468.0
59393,RD_F_J_20230228,2023-02-28,RD,F,J,421980.0,531.0
59394,RD_F_J_20230301,2023-03-01,RD,F,J,382980.0,574.0
59395,RD_F_J_20230302,2023-03-02,RD,F,J,477220.0,523.0


In [4]:
df = pd.concat([train,test]).reset_index(drop = True)

In [5]:
df.rename(columns={'supply(kg)':'supply', 'price(원/kg)':'price'},inplace=True)

In [8]:
def pre_all(train, test):
    print(f"전처리 전 train 크기 : {train.shape}")
    print(f"전처리 전 test 크기 : {test.shape}")
    print("=================전처리 중=================")

    # 합쳐서 전처리하기
    train["timestamp"] = pd.to_datetime(train["timestamp"])
    test["timestamp"] = pd.to_datetime(test["timestamp"])
    df = pd.concat([train,test]).reset_index(drop = True)

    df.rename(columns={'supply(kg)':'supply', 'price(원/kg)':'price'},inplace=True)

    #년/월/일 추가
    df['year']=df['timestamp'].dt.year
    df['month']=df['timestamp'].dt.month
    df['day']=df['timestamp'].dt.day

    #요일 추가
    df['week_day']=df['timestamp'].dt.weekday

    # 년-월 변수 추가 : year-month의 형태, 개월단위 누적값
    le = LabelEncoder()
    df["year_month"] = df["timestamp"].map(lambda x :str(x.year) + "-"+str(x.month))

    # 라벨 인코딩
    df["year_month"] = le.fit_transform(df["year_month"])


    # 주차 변수 추가
    df["week"] = df["timestamp"].map(lambda x: datetime.datetime(x.year, x.month, x.day).isocalendar()[1])

    # 주차 누적값
    week_list=[]
    for i in range(len(df['year'])) :
        if df['year'][i] == 2019 :
            week_list.append(int(df['week'][i]))
        elif df['year'][i] == 2020 :
            week_list.append(int(df['week'][i])+52)
        elif df['year'][i] == 2021 :
            week_list.append(int(df['week'][i])+52+53)
        elif df['year'][i] == 2022 :
            week_list.append(int(df['week'][i])+52+53+53)
        elif df['year'][i] == 2023 :
            week_list.append(int(df['week'][i])+52+53+53+52)
    df['week_num']= week_list

    # datetime 패키지에서 19년 12월 마지막주가 첫째주로 들어가는거 발견하여 수정
    df.loc[df['timestamp']=='2019-12-30','week_num']=52
    df.loc[df['timestamp']=='2019-12-31','week_num']=52


    # 공휴일 변수 추가
    def make_holi(x):
        kr_holi = holidays.KR()

        if x in kr_holi:
            return 1
        else:
            return 0

    df["holiday"] = df["timestamp"].map(lambda x : make_holi(x))


    # train, test 분리하기
    train = df[~df["price"].isnull()].sort_values("timestamp").reset_index(drop = True)
    test = df[df["price"].isnull()].sort_values("timestamp").reset_index(drop=True)


    print(f"전처리 후 train 크기 : {train.shape}")
    print(f"전처리 후 test 크기 : {test.shape}")

    return train, test

In [9]:
train_pre, test_pre = pre_all(train, test)

전처리 전 train 크기 : (59397, 7)
전처리 전 test 크기 : (1092, 5)
=================전처리 중=================
전처리 후 train 크기 : (59397, 15)
전처리 후 test 크기 : (1092, 15)


In [10]:
train_pre

,ID,timestamp,item,corporation,location,supply,price,year,month,day,week_day,year_month,week,week_num,holiday
0,TG_A_J_20190101,2019-01-01,TG,A,J,0.0,0.0,2019,1,1,1,0,1,1,1
1,CB_A_S_20190101,2019-01-01,CB,A,S,0.0,0.0,2019,1,1,1,0,1,1,1
2,RD_D_J_20190101,2019-01-01,RD,D,J,0.0,0.0,2019,1,1,1,0,1,1,1
3,BC_D_J_20190101,2019-01-01,BC,D,J,0.0,0.0,2019,1,1,1,0,1,1,1
4,CB_F_J_20190101,2019-01-01,CB,F,J,0.0,0.0,2019,1,1,1,0,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59392,CR_E_S_20230303,2023-03-03,CR,E,S,0.0,0.0,2023,3,3,4,50,9,219,0
59393,BC_A_S_20230303,2023-03-03,BC,A,S,3776.0,2875.0,2023,3,3,4,50,9,219,0
59394,CB_E_J_20230303,2023-03-03,CB,E,J,0.0,0.0,2023,3,3,4,50,9,219,0
59395,BC_D_J_20230303,2023-03-03,BC,D,J,1776.0,3059.0,2023,3,3,4,50,9,219,0


In [11]:
def remove_extreme_outliers(df, item, threshold):
    idx = df[(df["item"] == item) & (df["price"] > threshold)].index
    mean_price = df[(df["item"] == item) & (df["price"] != 0)]["price"].mean()
    df.loc[idx, "price"] = mean_price

In [15]:
# 감귤이 아닌 것
def filter_not_tg(df, item):
    return df[df["item"] != item]

In [40]:
# 인코딩
def encode_data(df, target_columns):
    return pd.get_dummies(df.sort_values(by=["timestamp"]).reset_index(drop=True), columns=target_columns)

In [16]:
remove_extreme_outliers(train_pre, "TG", 20000)
remove_extreme_outliers(train_pre, "RD", 5000)
remove_extreme_outliers(train_pre, "BC", 8000)
remove_extreme_outliers(train_pre, "CB", 2300)

In [24]:
# 감귤이 아닌 것 필터링
train_not_tg = filter_not_tg(train_pre, "TG")
test_not_tg = filter_not_tg(test_pre, "TG")

In [42]:
# Supply가 포함된 컬럼 삭제
def drop_columns_with_substring(df, substring):
    columns_to_drop = [col for col in df.columns if substring in col]
    return df.drop(columns=columns_to_drop)

In [44]:
# Supply가 포함된 컬럼 삭제 후 인코딩
target_columns = ["item", "corporation", "location"]
Xy = encode_data(drop_columns_with_substring(train_not_tg, "supply"), target_columns)
answer_notg = encode_data(test_not_tg, target_columns)

In [45]:
print(f"train의 컬럼 : {Xy.columns}")
print(f"test의 컬럼 : {answer_notg.columns}")

train의 컬럼 : Index(['ID', 'timestamp', 'price', 'year', 'month', 'day', 'week_day',
       'year_month', 'week', 'week_num', 'holiday', 'item_BC', 'item_CB',
       'item_CR', 'item_RD', 'corporation_A', 'corporation_B', 'corporation_C',
       'corporation_D', 'corporation_E', 'corporation_F', 'location_J',
       'location_S'],
      dtype='object')
test의 컬럼 : Index(['ID', 'timestamp', 'supply', 'price', 'year', 'month', 'day',
       'week_day', 'year_month', 'week', 'week_num', 'holiday', 'item_BC',
       'item_CB', 'item_CR', 'item_RD', 'corporation_A', 'corporation_B',
       'corporation_C', 'corporation_D', 'corporation_E', 'corporation_F',
       'location_J', 'location_S'],
      dtype='object')


In [62]:
from sklearn.model_selection import RandomizedSearchCV
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import VotingRegressor

In [65]:
## 전처리

# 극 이상치 제거
tg_idx = train_pre[(train_pre["item"]=="TG") & (train_pre["price"]>20000)].index
rd_idx = train_pre[(train_pre["item"]=="RD") & (train_pre["price"]>5000)].index
bc_idx = train_pre[(train_pre["item"]=="BC") & (train_pre["price"]>8000)].index
cb_idx = train_pre[(train_pre["item"]=="CB") & (train_pre["price"]>2300)].index

train_pre.loc[tg_idx,"price"] = train_pre[(train_pre["item"]=="TG") & (train_pre["price"]!=0)]["price"].mean()
train_pre.loc[rd_idx,"price"] = train_pre[(train_pre["item"]=="RD") & (train_pre["price"]!=0)]["price"].mean()
train_pre.loc[bc_idx,"price"] = train_pre[(train_pre["item"]=="BC") & (train_pre["price"]!=0)]["price"].mean()
train_pre.loc[cb_idx,"price"] = train_pre[(train_pre["item"]=="CB") & (train_pre["price"]!=0)]["price"].mean()


# 감귤이 아닌것
print(f"train의 컬럼 : {train_pre.columns}")
print(f"test의 컬럼 : {test_pre.columns}")

train_notg = train_pre[train_pre["item"] !="TG"]
test_notg = test_pre[test_pre["item"] != "TG"]


#인코딩
Xy = pd.get_dummies(train_notg.sort_values(by = ["timestamp"]).reset_index(drop=True).drop(columns = ["supply"]), columns = ["item","corporation","location"])
answer_notg = pd.get_dummies(test_notg.drop(columns = ["timestamp","supply","price"]), columns = [ "item","corporation","location"])
print(Xy.columns)

train의 컬럼 : Index(['ID', 'timestamp', 'item', 'corporation', 'location', 'supply', 'price',
       'year', 'month', 'day', 'week_day', 'year_month', 'week', 'week_num',
       'holiday'],
      dtype='object')
test의 컬럼 : Index(['ID', 'timestamp', 'item', 'corporation', 'location', 'supply', 'price',
       'year', 'month', 'day', 'week_day', 'year_month', 'week', 'week_num',
       'holiday'],
      dtype='object')
Index(['ID', 'timestamp', 'price', 'year', 'month', 'day', 'week_day',
       'year_month', 'week', 'week_num', 'holiday', 'item_BC', 'item_CB',
       'item_CR', 'item_RD', 'corporation_A', 'corporation_B', 'corporation_C',
       'corporation_D', 'corporation_E', 'corporation_F', 'location_J',
       'location_S'],
      dtype='object')


In [66]:
## 앙상블 모델 정의

cat = CatBoostRegressor(random_state = 2024,
                        n_estimators = 1000,
                        learning_rate = 0.01,
                        depth = 10,
                        l2_leaf_reg = 3,
                        metric_period = 1000)

xgb = XGBRegressor(n_estimators = 1000, random_state = 2024, learning_rate = 0.01, max_depth = 10)


# voting
vote_model = VotingRegressor(
    estimators =[("cat",cat), ("xgb", xgb)]
)

vote_model.fit(Xy.drop(columns = ["timestamp", "ID","price"]), Xy["price"])

pred = vote_model.predict(answer_notg.drop(columns = ["ID"]))
for idx in range(len(pred)):
    if pred[idx]<0:
        pred[idx]= 0
answer_notg["answer"] = pred

answer_notg[["ID","answer"]]

0:	learn: 934.3461033	total: 12.1ms	remaining: 12.1s
999:	learn: 376.6995081	total: 12.7s	remaining: 0us


,ID,answer
2,BC_B_J_20230304,2384.968678
4,BC_B_S_20230304,100.733655
6,BC_C_J_20230304,2293.269558
7,BC_A_S_20230304,2751.574472
10,BC_D_J_20230304,2776.219702
...,...,...
1085,RD_D_S_20230331,457.576344
1086,CR_C_J_20230331,1783.102829
1088,RD_D_J_20230331,398.828636
1089,CR_D_J_20230331,1822.411070


In [67]:
from sklearn.metrics import mean_absolute_error

# 예측값 계산
pred = vote_model.predict(Xy.drop(columns=["timestamp", "ID", "price"]))

# 음수 값 처리
pred = np.maximum(pred, 0)

# 실제값
true_values = Xy["price"]

# MAE 계산
mae = mean_absolute_error(true_values, pred)

print("Mean Absolute Error (MAE):", mae)


Mean Absolute Error (MAE): 122.97016223558009


In [68]:
from sklearn.metrics import mean_squared_error

# 예측값 계산
pred = vote_model.predict(Xy.drop(columns=["timestamp", "ID", "price"]))

# 음수 값 처리
pred = np.maximum(pred, 0)

# 실제값
true_values = Xy["price"]

# RMSE 계산
rmse = np.sqrt(mean_squared_error(true_values, pred))

print("Root Mean Squared Error (RMSE):", rmse)


Root Mean Squared Error (RMSE): 275.05159797602477


In [69]:
train_pre, test_pre = pre_all(train, test)

# 공휴일이지만 안쉬는 날 제외하기
no_holi = list(train_pre[(train_pre["item"] =="TG") &(train_pre["holiday"]==1) & (train_pre["price"]!=0)].groupby("timestamp").count().reset_index()["timestamp"])
noholi_idx = train_pre[train_pre["timestamp"].isin(no_holi)]["holiday"].index
for idx in noholi_idx:
    train_pre.loc[idx, "holiday"] = 0

# train 및 test 시간 순서로 정렬하기
train_tg = train_pre[train_pre["item"] == "TG"].sort_values(by = ["timestamp"]).reset_index(drop= True)
test_tg = test_pre[test_pre["item"] == "TG"].sort_values(by = ["timestamp"]).reset_index(drop= True)

Xy = pd.get_dummies(train_tg, columns = [ "item","corporation","location"]).drop(columns = ["supply"])
answer_tg1 = pd.get_dummies(test_tg, columns = [ "item","corporation","location"]).drop(columns = ["timestamp","supply","price"])
print(f"train의 컬럼 : {Xy.columns}")
print(f"test의 컬럼 : {answer_tg1.columns}")
Xy["price"] = np.sqrt(Xy["price"])

전처리 전 train 크기 : (59397, 7)
전처리 전 test 크기 : (1092, 5)
=================전처리 중=================
전처리 후 train 크기 : (59397, 15)
전처리 후 test 크기 : (1092, 15)
train의 컬럼 : Index(['ID', 'timestamp', 'price', 'year', 'month', 'day', 'week_day',
       'year_month', 'week', 'week_num', 'holiday', 'item_TG', 'corporation_A',
       'corporation_B', 'corporation_C', 'corporation_D', 'corporation_E',
       'location_J', 'location_S'],
      dtype='object')
test의 컬럼 : Index(['ID', 'year', 'month', 'day', 'week_day', 'year_month', 'week',
       'week_num', 'holiday', 'item_TG', 'corporation_A', 'corporation_B',
       'corporation_C', 'corporation_D', 'corporation_E', 'location_J',
       'location_S'],
      dtype='object')


In [70]:
# # 모델 정의
cat = CatBoostRegressor(random_state = 2024,
                        n_estimators = 1000,
                        learning_rate = 0.01,
                        depth = 10,
                        l2_leaf_reg = 3,
                        metric_period = 1000)

xgb = XGBRegressor(n_estimators = 1000, random_state = 2024, learning_rate = 0.01, max_depth = 10)


# voting
vote_model = VotingRegressor(
    estimators =[("cat",cat), ("xgb", xgb)]
)

vote_model.fit(Xy.drop(columns = ["timestamp", "ID","price"]), Xy["price"])

pred = vote_model.predict(answer_tg1.drop(columns = ["ID"]))
for idx in range(len(pred)):
    if pred[idx]<0:
        pred[idx]= 0
answer_tg1["answer"] = np.power(pred,2)

answer_tg1[["ID","answer"]]

0:	learn: 30.9664058	total: 13.6ms	remaining: 13.6s
999:	learn: 15.5181153	total: 11.7s	remaining: 0us


,ID,answer
0,TG_A_J_20230304,2563.261796
1,TG_E_S_20230304,3458.001764
2,TG_E_J_20230304,377.066719
3,TG_D_S_20230304,3662.783068
4,TG_D_J_20230304,607.878918
...,...,...
275,TG_D_J_20230331,2804.801999
276,TG_D_S_20230331,4860.908623
277,TG_A_S_20230331,5304.802147
278,TG_E_S_20230331,4859.787923


In [71]:
from sklearn.metrics import mean_squared_error

# 예측값 계산
pred = vote_model.predict(Xy.drop(columns=["timestamp", "ID", "price"]))

# 음수 값 처리
pred = np.maximum(pred, 0)

# 실제값
true_values = Xy["price"]

# RMSE 계산
rmse = np.sqrt(mean_squared_error(true_values, pred))

print("Root Mean Squared Error (RMSE):", rmse)

Root Mean Squared Error (RMSE): 12.098172279546729


In [72]:
from sklearn.model_selection import cross_val_score, KFold

# 예측값 계산
pred = vote_model.predict(Xy.drop(columns=["timestamp", "ID", "price"]))

# 음수 값 처리
pred = np.maximum(pred, 0)

# 실제값
true_values = Xy["price"]

# k-fold 교차 검증 설정
kfold = KFold(n_splits=5, shuffle=True, random_state=2024)

# 교차 검증 수행 및 RMSE 계산
rmse_scores = np.sqrt(-cross_val_score(vote_model, Xy.drop(columns=["timestamp", "ID", "price"]), true_values, scoring="neg_mean_squared_error", cv=kfold))

# 결과 출력
print("Cross-Validated RMSE Scores:", rmse_scores)
print("Mean RMSE:", np.mean(rmse_scores))


0:	learn: 31.0106097	total: 12.1ms	remaining: 12.1s
999:	learn: 15.4883333	total: 9.74s	remaining: 0us
0:	learn: 30.8871835	total: 10.5ms	remaining: 10.5s
999:	learn: 15.2709308	total: 11.2s	remaining: 0us
0:	learn: 31.0308251	total: 14.1ms	remaining: 14.1s
999:	learn: 15.2863358	total: 11.6s	remaining: 0us
0:	learn: 30.9086205	total: 12.6ms	remaining: 12.6s
999:	learn: 15.3324203	total: 11.9s	remaining: 0us
0:	learn: 31.0034726	total: 22ms	remaining: 22s
999:	learn: 15.4562047	total: 12.4s	remaining: 0us
Cross-Validated RMSE Scores: [16.32447666 16.92014942 16.68813997 16.87667956 16.46993766]
Mean RMSE: 16.655876653224997


In [73]:
train_tg2 = train_pre[train_pre["item"] =="TG"]
test_tg2 = test_pre[test_pre["item"] == "TG"]

Xy2 = pd.get_dummies(train_tg2.sort_values(by = ["timestamp", "corporation","location"]).reset_index(drop=True).drop(columns = ["item","supply"]), columns = [ "corporation","location"])
answer_tg2 = pd.get_dummies(test_tg2.drop(columns = ["timestamp","supply","price","item"]), columns = [ "corporation","location"])
print(Xy2.columns)

# 종속변수 루트값
Xy2["price"] = np.sqrt(Xy2["price"])

Index(['ID', 'timestamp', 'price', 'year', 'month', 'day', 'week_day',
       'year_month', 'week', 'week_num', 'holiday', 'corporation_A',
       'corporation_B', 'corporation_C', 'corporation_D', 'corporation_E',
       'location_J', 'location_S'],
      dtype='object')


In [2]:
# 모델 정의 및 훈련 예측
n_estimators =1000
lrs = 0.05
max_depths = 10
l2_leaf_reg = 3

cat = CatBoostRegressor(random_state = 2024,
                        n_estimators = n_estimators,
                        learning_rate = lrs,
                        depth = max_depths,
                        l2_leaf_reg = l2_leaf_reg,
                        metric_period = 1000)

cat.fit(Xy2.drop(columns = ["timestamp", "ID","price"]), Xy2["price"])

pred2 = cat.predict(answer_tg2.drop(columns = ["ID"]))
for idx in range(len(pred2)):
    if pred2[idx]<0:
        pred2[idx]= 0
answer_tg2["answer"] = np.power(pred2,2)

answer_tg2[["ID","answer"]]

NameError: name 'CatBoostRegressor' is not defined

In [1]:
# k-fold 교차 검증 설정
kfold = KFold(n_splits=5, shuffle=True, random_state=2024)

# 예측값 계산
pred2 = cat.predict(Xy2.drop(columns=["timestamp", "ID", "price"]))

# 음수 값 처리
pred2 = np.maximum(pred2, 0)

# 실제값
true_values2 = Xy2["price"]

# 교차 검증 수행 및 RMSE 계산
rmse_scores = np.sqrt(-cross_val_score(cat, Xy2.drop(columns=["timestamp", "ID", "price"]), true_values2, scoring="neg_mean_squared_error", cv=kfold))

# 결과 출력
print("Cross-Validated RMSE Scores:", rmse_scores)
print("Mean RMSE:", np.mean(rmse_scores))


NameError: name 'KFold' is not defined

In [9]:
total1 = pd.concat([answer_tg1[["ID","answer"]],answer_notg[["ID","answer"]]])
total2 = pd.concat([answer_tg2[["ID","answer"]],answer_notg[["ID","answer"]]])

NameError: name 'answer_tg1' is not defined